# Image augmentation and generation

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pickle
import glob
import random
from scipy import ndimage
from scipy.ndimage import zoom
import os

%matplotlib inline


In [ ]:
#data manipulation and add augmented data
def img_rot(image):
    rows,cols,ch = image.shape 
    center = (rows/2, cols/2)
    fact = random.randint(-1,1)
    return cv2.warpAffine(image, cv2.getRotationMatrix2D(center, fact*5, 1), (rows, cols))

def img_shift(image):
    rows,cols,ch = image.shape 
    fact1 = random.randint(-10,10)
    fact2 = random.randint(-10,10)
    return ndimage.shift(image,[fact1,fact2,0])

def clipped_zoom(img, zoom_factor, **kwargs): # from stackoverflow ali_m

    h, w = img.shape[:2]

    # width and height of the zoomed image
    zh = int(np.round(zoom_factor * h))
    zw = int(np.round(zoom_factor * w))

    # for multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # zooming out
    if zoom_factor < 1:
        # bounding box of the clip region within the output array
        top = (h - zh) // 2
        left = (w - zw) // 2
        # zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # zooming in
    elif zoom_factor > 1:
        # bounding box of the clip region within the input array
        top = (zh - h) // 2
        left = (zw - w) // 2
        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)
        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # if zoom_factor == 1, just return the input array
    else:
        out = img
    return out

def img_zoom(image):     
    zoom_f = float(0.85 + (random.randint(0,3)/10))
    return clipped_zoom(image, zoom_f) 

def img_shear(image):
    rows,cols,ch = image.shape
    shear_fact = random.randint(5,15)
    # Shear
    scr = np.float32([[5,5],[20,5],[5,20]])

    p_dst1 = 5+shear_fact*np.random.uniform()-shear_fact/2
    p_dst2 = 20+shear_fact*np.random.uniform()-shear_fact/2

    dst = np.float32([[p_dst1,5],[p_dst2,p_dst1],[5,p_dst2]])

    return cv2.warpAffine(image,cv2.getAffineTransform(scr,dst),(cols,rows))
###################################################################################################


In [ ]:
# Read in cars and notcars
#    images = glob.glob('data_base/*.jpeg')
images = glob.glob('data_base_long/*.png')
cars = []
notcars = []
for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)

print('notcars: ', len(notcars))
print('cars: ', len(cars))
count_car = 1
count_notcar = 1
for file in cars:
    # Read in each one by one
#        image = mpimg.imread(file)# for jpg,    png file 0-1, jpg file 0-255
    image = cv2.imread(file)# for png,    png file 0-255
    for k in range(4):
        if k == 0:
            aug_img = img_rot(image)
        elif k == 1:
            aug_img = img_shift(image)
        elif k == 2:
            aug_img = img_zoom(image)
        elif k == 3:
            aug_img = img_shear(image)
        f_name = 'car__0' + str(count_car) + '.png'
        daf_name = 'data_aug_01/' + f_name
        cv2.imwrite(daf_name,aug_img)
        count_car +=1

for file in notcars:
    # Read in each one by one
#        image = mpimg.imread(file)# for jpg,    png file 0-1, jpg file 0-255
    image = cv2.imread(file)# for png,    png file 0-255
    for k in range(4):
        if k == 0:
            aug_img = img_rot(image)
        elif k == 1:
            aug_img = img_shift(image)
        elif k == 2:
            aug_img = img_zoom(image)
        elif k == 3:
            aug_img = img_shear(image)
        f_name = 'image__0' + str(count_notcar) + '.png'
        daf_name = 'data_aug_01/' + f_name
        cv2.imwrite(daf_name,aug_img)
        count_notcar +=1

In [ ]:
# Read in cars and notcars
images = glob.glob('data_base_jpg_short/*.jpeg')

for img in images:
    image = mpimg.imread(img)
    img_name = os.path.splitext(os.path.basename(img))[0]
    f_name = img_name + '.png'
    daf_name = 'data_base_short/' + f_name
    cv2.imwrite(daf_name,image)    

In [ ]:
def pre_process_image(image):

    #image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    #image = image[:,:,0]
    image[:,:,0] = cv2.equalizeHist(image[:,:,0])
    image[:,:,1] = cv2.equalizeHist(image[:,:,1])
    image[:,:,2] = cv2.equalizeHist(image[:,:,2])
    image = image/255.-.5
    #image = cv2.resize(image, (img_resize,img_resize),interpolation = cv2.INTER_CUBIC)

    return image



In [ ]:
input_org = []

for imgname in os.listdir("./data_base_long/"):
    image_org = cv2.imread('data_base_long/' + imgname)
    input_org.append(image_org)

    
fig, ax = plt.subplots(2,5, sharex=True, figsize=(15,15))    
for k, image_pic in enumerate(input_org):
    plt.subplot(2,5,k+1) 
    plt.imshow(image_pic)
    plot_title = (k)
    plt.title(plot_title )
    
plt.tight_layout()   
plt.show()